# `ProteinWorkshop` Tutorial, Part 2 - Customizing an Existing Dataset
![Datasets](../docs/source/_static/box_datasets.png)

## Repurpose an existing dataset within the `ProteinWorkshop`

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

Welcome back to the `ProteinWorkshop` tutorial series!

To reuse existing data components available in the `ProteinWorkshop`, you can follow the following 5-step procedure:

1. Use an existing dataset as either a pre-training or fine-tuning corpus, with or without full-atom context
2. Load the requested dataset using the designed config
3. Either pre-train or fine-tune a model using the selected dataset
4. Reconfigure the selected dataset to use e.g., side-chain atom context
5. Verify that e.g., side-chain torsions are now available as feature inputs

### 1. Use an existing dataset as either a pre-training or fine-tuning corpus, with or without full-atom context

One can switch out the dataset for any other available option by replacing the value of `dataset` in `overrides`:

`cfg = hydra.compose("template", overrides=["encoder=schnet", "task=inverse_folding", "dataset=cath", "features=ca_angles", "+aux_task=none"], return_hydra_config=True)`

In [2]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")

GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=schnet",
        "task=inverse_folding",
        "dataset=cath",
        "features=ca_angles",
        "+aux_task=none",
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

[01/26/25 14:30:32] INFO     DATA_PATH: /Users/dricpro/PycharmProjects/Topotein/data                ]8;id=3174;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=508808;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/constants.py#39\39]8;;\

[01/26/25 14:30:34] INFO     Hydra initialised at                                                    ]8;id=469773;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=40194;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop               
                             /config.                                                                              

### 2. Load the requested dataset using the designed config

In [3]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

datamodule = hydra.utils.instantiate(cfg.dataset.datamodule)
datamodule.setup("fit")
dl = datamodule.train_dataloader()

for i in dl:
    print(i)
    break

[01/26/25 14:30:48] WARNING  You are launching an inverse folding experiment with amino_acid_one_hot  ]8;id=941170;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=589578;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#114\114]8;;\
                             as a feature. This will be removed.                                                   

                    DEBUG    Requested CPUs: 16. Avialable CPUs (physical): 12. Requested CPU count   ]8;id=75771;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=128841;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#312\312]8;;\
                             will therefore be set to maximum number ofavailable physical cores.                   
                             NOTE: It is recommended to use N-1cores or less to avoid memory flush                 
                             overheads.                                                                            

                    DEBUG    CUDA available: False                                                    ]8;id=502789;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=150994;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#247\247]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=281107;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=961864;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#248\248]8;;\

                    DEBUG    GPU count set to: 0                                                      ]8;id=928938;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=338262;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#253\253]8;;\

...

                    INFO     All structures already processed and overwrite=False. Skipping download.   ]8;id=705523;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/datasets/base.py\base.py]8;;\:]8;id=102233;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/datasets/base.py#327\327]8;;\

                    INFO     All structures already processed and overwrite=False. Skipping download.   ]8;id=454933;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/datasets/base.py\base.py]8;;\:]8;id=879272;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/datasets/base.py#362\362]8;;\

                    INFO     Reading data into memory                                                   ]8;id=56646;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/datasets/base.py\base.py]8;;\:]8;id=335767;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/datasets/base.py#337\337]8;;\

100%|██████████| 607/607 [00:00<00:00, 3165.19it/s]


DataProteinBatch(fill_value=1e-05, atom_list=[37], coords=[6364, 37, 3], residues=[32], id=[32], residue_id=[32], residue_type=[6364], chains=[6364], x=[6364], amino_acid_one_hot=[6364, 23], seq_pos=[6364, 1], batch=[6364], ptr=[33])


### 3. Either pre-train or fine-tune a model using the selected dataset

In [8]:
from proteinworkshop.finetune import finetune
from proteinworkshop.train import train_model

train_model(cfg)  # Pre-train a model using the selected data
# finetune(cfg)  # Fine-tune a model using the selected data

/opt/anaconda3/envs/Topotein/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning:

PEP 484 type hint typing.Tuple[torch.Tensor, torch.Tensor] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations

/opt/anaconda3/envs/Topotein/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning:

PEP 484 type hint typing.Callable deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/e

[01/26/25 16:09:33] INFO     Instantiating datamodule: <proteinworkshop.datasets.cath.CATHDataModule... ]8;id=896664;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=987468;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/train.py#99\99]8;;\

                    INFO     Instantiating callbacks...                                                ]8;id=166878;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=837528;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/train.py#106\106]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.ModelCheckpoint>   ]8;id=1185;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=35855;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.EarlyStopping>     ]8;id=248769;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=72738;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

...

                    INFO     Instantiating callback <lightning.pytorch.callbacks.EarlyStopping>     ]8;id=935800;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=924324;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

                    INFO     Instantiating loggers...                                                  ]8;id=545720;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=702368;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/train.py#111\111]8;;\

                    INFO     Instantiating logger <lightning.pytorch.loggers.csv_logs.CSVLogger>      ]8;id=193480;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/loggers.py\loggers.py]8;;\:]8;id=825667;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/loggers.py#31\31]8;;\

                    INFO     Instantiating trainer...                                                  ]8;id=750405;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=447387;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/train.py#114\114]8;;\

InstantiationException: Error in call to target 'lightning.pytorch.trainer.trainer.Trainer':
MisconfigurationException('`Trainer(devices=0)` value is not a valid input using mps accelerator.')
full_key: trainer

### 4. Reconfigure the selected dataset to use e.g., side-chain atom context

In [6]:
version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")

GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=schnet",
        "task=sequence_denoising",
        "dataset=cath",
        "features=ca_sc",
        "+aux_task=none",
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

[01/26/25 14:32:33] INFO     Hydra singleton cleared and ready to re-initialise.                     ]8;id=834942;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=443049;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/notebook.py#66\66]8;;\

                    INFO     Hydra initialised at                                                    ]8;id=347205;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=839408;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop               
                             /config.                                                                              

### 5. Verify that e.g., side-chain torsions are now available as feature inputs

In [7]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

datamodule = hydra.utils.instantiate(cfg)
datamodule.setup("fit")
dl = datamodule.train_dataloader()

for i in dl:
    print(i)
    break

[01/26/25 14:32:35] DEBUG    Requested CPUs: 16. Avialable CPUs (physical): 12. Requested CPU count   ]8;id=167728;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=356436;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#312\312]8;;\
                             will therefore be set to maximum number ofavailable physical cores.                   
                             NOTE: It is recommended to use N-1cores or less to avoid memory flush                 
                             overheads.                                                                            

                    DEBUG    CUDA available: False                                                    ]8;id=524790;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=670361;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#247\247]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=116477;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=353696;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#248\248]8;;\

                    DEBUG    GPU count set to: 0                                                      ]8;id=459724;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=65648;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#253\253]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=718357;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=685029;file:///Users/dricpro/PycharmProjects/Topotein/ProteinWorkshop/proteinworkshop/configs/config.py#164\164]8;;\

ReadonlyConfigError: Cannot set value of read-only config node

### 6. Wrapping up

Have any additional questions about using the existing data components provided in the `ProteinWorkshop`? [Create a new issue](https://github.com/a-r-j/ProteinWorkshop/issues/new/choose) on our [GitHub repository](https://github.com/a-r-j/ProteinWorkshop). We would be happy to work with you to leverage the full power of the repository!